# model nn keras

* [keras guide](https://keras.io/getting-started/functional-api-guide/)

In [1]:

import os
import numpy as np
import tensorflow as tf
import keras 
from keras.layers import Input, Dense  
from keras.models import Model
from keras.utils import to_categorical
from keras import backend as K
from keras.callbacks import  EarlyStopping, ModelCheckpoint
from helper import read_mnist_data



print(tf.__version__)
print(keras.__version__)

/Users/lywang/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


1.10.0
2.2.2


Using TensorFlow backend.


In [2]:
DATA_PATH = '/Users/lywang/Data/mnist'
image_train, label_train, image_test, label_test = read_mnist_data(DATA_PATH)
print(image_train.shape, label_train.shape )
print(image_test.shape, label_test.shape )

(60000, 784) (60000, 1)
(10000, 784) (10000, 1)


In [3]:
label_train_onehot = to_categorical(label_train)
label_test_onehot = to_categorical(label_test)

In [4]:
MODEL_DIR = 'model_checkpoints/'

In [5]:
# restart training everytime
K.clear_session()

model_path = os.path.join(MODEL_DIR, 'keras.{epoch:02d}-{val_loss:.2f}-{val_acc:.2f}.hdf5')

# construct
inputs = Input(shape=(784,))
x = Dense(128, activation='relu')(inputs)
x = Dense(64, activation='relu')(x)
x = Dense(32, activation='relu')(x)
predictions = Dense(10, activation='softmax')(x)
model = Model(inputs=inputs, outputs=predictions)

model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

callback_list = [EarlyStopping(monitor='val_loss', patience=5),
                 ModelCheckpoint(model_path, monitor='val_loss', 
                                 verbose=1, save_best_only=True, 
                                 period=1)
                ]
model.fit(image_train, label_train_onehot, 
          batch_size=500, 
          epochs=100, 
          shuffle=True,
          validation_split=0.25,
          callbacks=callback_list)  # starts training

Train on 45000 samples, validate on 15000 samples
Epoch 1/100
45000/45000 [==============================] - 25s 561us/step - loss: 5.6121 - acc: 0.6321 - val_loss: 3.2511 - val_acc: 0.7754

Epoch 00001: val_loss improved from inf to 3.25107, saving model to model_checkpoints/keras.01-3.25-0.78.hdf5
Epoch 2/100
45000/45000 [==============================] - 18s 402us/step - loss: 2.5514 - acc: 0.8280 - val_loss: 2.5776 - val_acc: 0.8281

Epoch 00002: val_loss improved from 3.25107 to 2.57760, saving model to model_checkpoints/keras.02-2.58-0.83.hdf5
Epoch 3/100
45000/45000 [==============================] - 18s 401us/step - loss: 2.3092 - acc: 0.8451 - val_loss: 2.2990 - val_acc: 0.8473

Epoch 00003: val_loss improved from 2.57760 to 2.29898, saving model to model_checkpoints/keras.03-2.30-0.85.hdf5
Epoch 4/100
45000/45000 [==============================] - 19s 417us/step - loss: 2.2075 - acc: 0.8526 - val_loss: 2.3119 - val_acc: 0.8477

Epoch 00004: val_loss did not improve from 2.298


Epoch 00036: val_loss did not improve from 0.43508
Epoch 37/100
45000/45000 [==============================] - 19s 432us/step - loss: 0.1630 - acc: 0.9869 - val_loss: 0.4758 - val_acc: 0.9636

Epoch 00037: val_loss did not improve from 0.43508


## load model for prediction


In [7]:
loaded_model = keras.models.load_model(os.path.join(MODEL_DIR, 'keras_128x64x32.32-0.44-0.97.hdf5'))

In [8]:
prediction = loaded_model.predict(image_test)
prediction = np.argmax(prediction, axis=1)

In [9]:
test_acc = sum(prediction == np.squeeze(label_test)) / len(prediction)
print('test acc: {} (error rate {:.2f}%) '.format(test_acc, (1-test_acc)*100))

test acc: 0.9704 (error rate 2.96%) 


record: 

64x64x32: val_acc 0.9469; test_acc 0.9506

128x64x32: val_acc 0.9869; test_acc 0.9704
